In [76]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os

In [77]:
base_path_right = "/home/finn/.cache/huggingface/lerobot/F-Fer/ur-task3-0/meta/episodes/chunk-000/"
base_path = "/home/finn/.cache/huggingface/lerobot/F-Fer/ur-tasks-merged/meta/episodes/chunk-000/"

In [78]:
def get_parquet_files(base_path):
    parquet_files = [f for f in os.listdir(base_path) if f.endswith('.parquet')]
    parquet_files = [os.path.join(base_path, f) for f in sorted(parquet_files)]
    return parquet_files

parquet_files_right = get_parquet_files(base_path_right)
parquet_files = get_parquet_files(base_path)

print(parquet_files)


['/home/finn/.cache/huggingface/lerobot/F-Fer/ur-tasks-merged/meta/episodes/chunk-000/file-000.parquet']


In [79]:
def get_schema_dict(path):
    try:
        # Read schema only
        schema = pq.read_schema(path)
        # Create a dict of {column_name: string_representation_of_type}
        return {name: str(schema.field(name).type) for name in schema.names}
    except Exception as e:
        print(f"Error reading {path}: {e}")
        return {}


In [80]:
# Compare the schemas of of two files
def compare_schemas(path1, path2):
    schema1 = get_schema_dict(path1)
    schema2 = get_schema_dict(path2)
    all_cols = sorted(set(schema1.keys()) | set(schema2.keys()))

    for col in all_cols:
        type1 = schema1.get(col, "MISSING")
        type2 = schema2.get(col, "MISSING")

        if type1 != type2:
            marker = "Mismatch -> "
            print(f"{path2}: {marker}{col:<50} | {type1:<30} | {type2:<30}")
        else:
            marker = ""

compare_schemas(parquet_files_right[0], parquet_files[0])

In [81]:
for i in range(len(parquet_files)):
    compare_schemas(parquet_files_right[0], parquet_files[i])

## Explore merged dataset

In [99]:
# Force pandas to show all columns
pd.set_option('display.max_columns', None)


In [105]:
base_path = "/home/finn/.cache/huggingface/lerobot/F-Fer/ur-tasks-merged/meta/episodes/chunk-000/"
parquet_files = get_parquet_files(base_path)

print(parquet_files)

# Only one file
meta_file = parquet_files[0]

df_merged = pd.read_parquet(meta_file)
df_right = pd.read_parquet(parquet_files_right[0])

# Sort columns by name
df_merged = df_merged.reindex(sorted(df_merged.columns), axis=1)
df_right = df_right.reindex(sorted(df_right.columns), axis=1)


['/home/finn/.cache/huggingface/lerobot/F-Fer/ur-tasks-merged/meta/episodes/chunk-000/file-000.parquet']


In [106]:
# Show in table
df_merged.head(2)

,data/chunk_index,data/file_index,dataset_from_index,dataset_to_index,episode_index,length,meta/episodes/chunk_index,meta/episodes/file_index,stats/action/count,stats/action/max,stats/action/mean,stats/action/min,stats/action/q01,stats/action/q10,stats/action/q50,stats/action/q90,stats/action/q99,stats/action/std,stats/episode_index/count,stats/episode_index/max,stats/episode_index/mean,stats/episode_index/min,stats/episode_index/q01,stats/episode_index/q10,stats/episode_index/q50,stats/episode_index/q90,stats/episode_index/q99,stats/episode_index/std,stats/frame_index/count,stats/frame_index/max,stats/frame_index/mean,stats/frame_index/min,stats/frame_index/q01,stats/frame_index/q10,stats/frame_index/q50,stats/frame_index/q90,stats/frame_index/q99,stats/frame_index/std,stats/index/count,stats/index/max,stats/index/mean,stats/index/min,stats/index/q01,stats/index/q10,stats/index/q50,stats/index/q90,stats/index/q99,stats/index/std,stats/observation.images.zed2i_left/count,stats/observation.images.zed2i_left/max,stats/observation.images.zed2i_left/mean,stats/observation.images.zed2i_left/min,stats/observation.images.zed2i_left/q01,stats/observation.images.zed2i_left/q10,stats/observation.images.zed2i_left/q50,stats/observation.images.zed2i_left/q90,stats/observation.images.zed2i_left/q99,stats/observation.images.zed2i_left/std,stats/observation.images.zed2i_right/count,stats/observation.images.zed2i_right/max,stats/observation.images.zed2i_right/mean,stats/observation.images.zed2i_right/min,stats/observation.images.zed2i_right/q01,stats/observation.images.zed2i_right/q10,stats/observation.images.zed2i_right/q50,stats/observation.images.zed2i_right/q90,stats/observation.images.zed2i_right/q99,stats/observation.images.zed2i_right/std,stats/observation.images.zedm_left/count,stats/observation.images.zedm_left/max,stats/observation.images.zedm_left/mean,stats/observation.images.zedm_left/min,stats/observation.images.zedm_left/q01,stats/observation.images.zedm_left/q10,stats/observation.images.zedm_left/q50,stats/observation.images.zedm_left/q90,stats/observation.images.zedm_left/q99,stats/observation.images.zedm_left/std,stats/observation.images.zedm_right/count,stats/observation.images.zedm_right/max,stats/observation.images.zedm_right/mean,stats/observation.images.zedm_right/min,stats/observation.images.zedm_right/q01,stats/observation.images.zedm_right/q10,stats/observation.images.zedm_right/q50,stats/observation.images.zedm_right/q90,stats/observation.images.zedm_right/q99,stats/observation.images.zedm_right/std,stats/observation.state/count,stats/observation.state/max,stats/observation.state/mean,stats/observation.state/min,stats/observation.state/q01,stats/observation.state/q10,stats/observation.state/q50,stats/observation.state/q90,stats/observation.state/q99,stats/observation.state/std,stats/task_index/count,stats/task_index/max,stats/task_index/mean,stats/task_index/min,stats/task_index/q01,stats/task_index/q10,stats/task_index/q50,stats/task_index/q90,stats/task_index/q99,stats/task_index/std,stats/timestamp/count,stats/timestamp/max,stats/timestamp/mean,stats/timestamp/min,stats/timestamp/q01,stats/timestamp/q10,stats/timestamp/q50,stats/timestamp/q90,stats/timestamp/q99,stats/timestamp/std,tasks,videos/observation.images.zed2i_left/chunk_index,videos/observation.images.zed2i_left/file_index,videos/observation.images.zed2i_left/from_timestamp,videos/observation.images.zed2i_left/to_timestamp,videos/observation.images.zed2i_right/chunk_index,videos/observation.images.zed2i_right/file_index,videos/observation.images.zed2i_right/from_timestamp,videos/observation.images.zed2i_right/to_timestamp,videos/observation.images.zedm_left/chunk_index,videos/observation.images.zedm_left/file_index,videos/observation.images.zedm_left/from_timestamp,videos/observation.images.zedm_left/to_timestamp,videos/observation.images.zedm_right/chunk_index,videos/observation.images.zedm_right/file_index,videos/observation.images.zedm_right/from_tim

In [107]:
# Show head of right dataset
df_right.head(2)

,data/chunk_index,data/file_index,dataset_from_index,dataset_to_index,episode_index,length,meta/episodes/chunk_index,meta/episodes/file_index,stats/action/count,stats/action/max,stats/action/mean,stats/action/min,stats/action/q01,stats/action/q10,stats/action/q50,stats/action/q90,stats/action/q99,stats/action/std,stats/episode_index/count,stats/episode_index/max,stats/episode_index/mean,stats/episode_index/min,stats/episode_index/q01,stats/episode_index/q10,stats/episode_index/q50,stats/episode_index/q90,stats/episode_index/q99,stats/episode_index/std,stats/frame_index/count,stats/frame_index/max,stats/frame_index/mean,stats/frame_index/min,stats/frame_index/q01,stats/frame_index/q10,stats/frame_index/q50,stats/frame_index/q90,stats/frame_index/q99,stats/frame_index/std,stats/index/count,stats/index/max,stats/index/mean,stats/index/min,stats/index/q01,stats/index/q10,stats/index/q50,stats/index/q90,stats/index/q99,stats/index/std,stats/observation.images.zed2i_left/count,stats/observation.images.zed2i_left/max,stats/observation.images.zed2i_left/mean,stats/observation.images.zed2i_left/min,stats/observation.images.zed2i_left/q01,stats/observation.images.zed2i_left/q10,stats/observation.images.zed2i_left/q50,stats/observation.images.zed2i_left/q90,stats/observation.images.zed2i_left/q99,stats/observation.images.zed2i_left/std,stats/observation.images.zed2i_right/count,stats/observation.images.zed2i_right/max,stats/observation.images.zed2i_right/mean,stats/observation.images.zed2i_right/min,stats/observation.images.zed2i_right/q01,stats/observation.images.zed2i_right/q10,stats/observation.images.zed2i_right/q50,stats/observation.images.zed2i_right/q90,stats/observation.images.zed2i_right/q99,stats/observation.images.zed2i_right/std,stats/observation.images.zedm_left/count,stats/observation.images.zedm_left/max,stats/observation.images.zedm_left/mean,stats/observation.images.zedm_left/min,stats/observation.images.zedm_left/q01,stats/observation.images.zedm_left/q10,stats/observation.images.zedm_left/q50,stats/observation.images.zedm_left/q90,stats/observation.images.zedm_left/q99,stats/observation.images.zedm_left/std,stats/observation.images.zedm_right/count,stats/observation.images.zedm_right/max,stats/observation.images.zedm_right/mean,stats/observation.images.zedm_right/min,stats/observation.images.zedm_right/q01,stats/observation.images.zedm_right/q10,stats/observation.images.zedm_right/q50,stats/observation.images.zedm_right/q90,stats/observation.images.zedm_right/q99,stats/observation.images.zedm_right/std,stats/observation.state/count,stats/observation.state/max,stats/observation.state/mean,stats/observation.state/min,stats/observation.state/q01,stats/observation.state/q10,stats/observation.state/q50,stats/observation.state/q90,stats/observation.state/q99,stats/observation.state/std,stats/task_index/count,stats/task_index/max,stats/task_index/mean,stats/task_index/min,stats/task_index/q01,stats/task_index/q10,stats/task_index/q50,stats/task_index/q90,stats/task_index/q99,stats/task_index/std,stats/timestamp/count,stats/timestamp/max,stats/timestamp/mean,stats/timestamp/min,stats/timestamp/q01,stats/timestamp/q10,stats/timestamp/q50,stats/timestamp/q90,stats/timestamp/q99,stats/timestamp/std,tasks,videos/observation.images.zed2i_left/chunk_index,videos/observation.images.zed2i_left/file_index,videos/observation.images.zed2i_left/from_timestamp,videos/observation.images.zed2i_left/to_timestamp,videos/observation.images.zed2i_right/chunk_index,videos/observation.images.zed2i_right/file_index,videos/observation.images.zed2i_right/from_timestamp,videos/observation.images.zed2i_right/to_timestamp,videos/observation.images.zedm_left/chunk_index,videos/observation.images.zedm_left/file_index,videos/observation.images.zedm_left/from_timestamp,videos/observation.images.zedm_left/to_timestamp,videos/observation.images.zedm_right/chunk_index,videos/observation.images.zedm_right/file_index,videos/observation.images.zedm_right/from_tim